In [1]:
import psycopg2
# Connect to an existing database
conn = psycopg2.connect("dbname='teste_03' user='postgres' host='localhost' password='severo' port='5432'")

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a command: this creates a new table
cur.execute("select m.tid,m.time,m.edge_id,m.longitude,m.latitude from matched_tracks m order by m.tid,m.time;")
matched_tracks = cur.fetchall()
cur.execute("select distinct edge_id,min(longitude),min(latitude) from matched_tracks group by edge_id order by edge_id asc;")
egde_ids = cur.fetchall()
cur.execute("select i.tid,i.start_time,i.end_time,EXTRACT(EPOCH FROM (i.end_time::timestamp - i.start_time::timestamp)) from ib_stops_carlos i order by i.tid, start_time;")
stops = cur.fetchall()
cur.execute("select min(m.longitude),min(m.latitude),max(m.latitude),max(m.longitude) from matched_tracks m;")
p =cur.fetchall()
# Pass data to fill a query placeholders and let Psycopg perform
# the correct conversion (no more SQL injections!)
#>>> cur.execute("INSERT INTO test (num, data) VALUES (%s, %s)",
#...      (100, "abc'def"))

# Query the database and obtain data as Python objects
#>>> cur.execute("SELECT * FROM test;")
#>>> cur.fetchone()
#(1, 100, "abc'def")

# Make the changes to the database persistent
#>>> conn.commit()

# Close communication with the database
cur.close()

conn.close()

In [3]:
# função que verificar se um ponto é um stop.
def verifica(ponto,stops):
    for stop in stops:
        if ponto[0]==stop[0] and ponto[1]==stop[1]:
            return stop[3]
    return -1 
pd = list()
for i in p:
    pd.extend(i)

In [4]:
# Encrontando as trajetorias.
trajetorias = list()
trajetoria = list()
taxi_id = 1

for ponto in matched_tracks:
    if ponto[0]==taxi_id:
        diferenca=verifica([ponto[0],ponto[1]],stops)
        if diferenca==-1:
            trajetoria.append(ponto)
        else:
            trajetoria.append(ponto)
            if diferenca > 120:
                trajetorias.append(trajetoria)
                trajetoria = list()
    else:
        trajetorias.append(trajetoria)
        trajetoria = list()
        trajetoria.append(ponto)
        taxi_id=taxi_id+1     

In [5]:
# Criando dicionário para mapeamento dos egde_ids para os indices da matriz m:
dicionario1=dict()
indice=0
for egde_id in egde_ids:
    dicionario1[egde_id[0]]=indice
    indice=indice+1
# Criando dicionário para o mapeamento dos indices da matriz para os egdes_ids:
dicionario2 = dict()
for i in egde_ids:
    d=dicionario1[i[0]]
    dicionario2[d]=i

In [6]:
# Criando a matriz de transições:
import numpy as np
k=len(dicionario1)
m =np.zeros( (k,k),dtype=np.int32)

In [7]:
# Contando quantas transições exites de cada egde_id para outro:
for traj in trajetorias:
    c = len(traj)
    for i in range(0,c-1):
        if traj[i][2] != traj[i+1][2]:
            d1=dicionario1[traj[i][2]]
            d2=dicionario1[traj[i+1][2]]
            m[d1][d2]=m[d1][d2]+1

In [8]:
# Contando a quantidade de celulas não nulas.
count=0
for n in m:
    for i in n:
        if i!= 0:
            count=count+1

In [9]:
# Calculando as dimensões da matriz para mapeamento.
import math
raiz=math.ceil(math.sqrt(count))
# Criando a matriz para o mapeamento.
n =np.zeros( (raiz,raiz),dtype=np.int32)

In [10]:
# Mapeamento de uma matriz para outra.
dicionario3=dict()
l=0
c=0
for i in range(0,k):
    for j in range(0,k):
        if m[i][j] !=0:
            n[l][c]=m[i][j]
            dicionario3[(l,c)]=(i,j)
            c=c+1
            if c==raiz:
                c=0
                l=l+1

In [11]:
def not_marked(m,a):
    for n in m:
        for i in n:
            if i > 0 and i>=a:
                return True
    return False

def maximo(m,c):
    l=0
    f=0
    for i in range(0,c):
        for j in range(0,c):
            if m[i][j]>m[l][f]:
                l=i
                f=j
    return [l,f]

def marking(a):
    if a>0:
        return -1*a
    return a

def not_marked_2(m,d,f,a,e,c):
    u=-1
    minimo=100000
    for i in range(0,c):
        if m[f][i]>=a:
            t = m[d][f]
            if t<0:
                t=t*(-1)
            if abs(t-m[f][i]) < minimo and abs(t-m[f][i])<=e:
                minimo = abs(t-m[f][i])
                u=i
    return u

def not_marked_3(m,d,f,a,e,c):
    u=-1
    minimo=1000000
    for i in range(0,c):
        if m[i][d]>=a:
            t = m[d][f]
            if t<0:
                t=t*(-1)
            if abs(t-m[u][d])<minimo and abs(t-m[u][d])<=e:
                minimo=abs(t-m[u][d])
                u=i
    return u

In [12]:
def netscan_phase_1(dicionario3,c,dicionario2,m,e,a):
    O=set()
    k=0
    C=list()
    while not_marked(m,a):
        indices=maximo(m,c)
        d=indices[0]
        f=indices[1]
        tupla=dicionario3[(d,f)]
        Sd=dicionario2[tupla[0]]
        Sf=dicionario2[tupla[1]]
        C.append([Sd,Sf])
        m[d][f]=marking(m[d][f])
        u=not_marked_2(m,d,f,a,e,c)
        while u>-1:
            tupla=dicionario3[(f,u)]
            Su=dicionario2[tupla[1]]
            C[k].append(Su)
            m[f][u]=marking(m[f][u])
            d=f
            f=u
            u=not_marked_2(m,d,f,a,e,c)
        u=not_marked_3(m,d,f,a,e,c)
        while u>-1:
            tupla=dicionario3[(u,d)]
            Su=dicionario2[tupla[0]]
            C[k] = [Su]+C[k]
            m[u][d]=marking(m[u][d])
            f=d
            d=u
            u=not_marked_3(m,d,f,a,e,c)
        k=k+1
    return C

In [13]:
C=netscan_phase_1(dicionario3,raiz,dicionario2,n,1,15)

In [14]:
def overlaps(tr,c):
    for i in tr:
        for j in c:
            if i[2]==j[0]:
                return True
    return False

def intersecao(c,tr):
    i=0
    j=0
    Sc=list()
    intersec=list()
    while i<len(c):
        while j<len(tr):
            if c[i][0]==tr[j][2]:
                #print("This is tr[",j,"] : ", tr[j])
                intersec.append(tr[j])
                i=i+1;
                j=j+1;
                if i>=len(c):
                    if len(intersec)>0:
                        Sc.extend(intersec)
                    break
            else:
                j=j+1
                if len(intersec)>0:
                    Sc.extend(intersec)
               # print("intersec",intersec)
                intersec=list()
        if j>=len(tr):
            break
        i=i+1
    return Sc

def distancia(x1,y1,x2,y2):
     return math.sqrt(math.pow(x1-x2,2)+math.pow(y1-y2,2))

In [15]:
def netscan_phase2(O,traj,s):
    Cluster=list()
    count =0
    for c in O:
        clus = list()
        for tr in traj:
            Sc=list()
            if overlaps(tr,c):
                Sc = intersecao(c,tr)
                if len(Sc) >0:
                    count=count+1
                    Lc = 0
                    Lc = Lc + (distancia(Sc[0][3],Sc[0][4],Sc[len(Sc)-1][3],Sc[len(Sc)-1][4])/distancia(pd[0],pd[1],pd[2],pd[3]))
                    Sim = Lc/(distancia(c[0][1],c[0][2],c[len(c)-1][1],c[len(c)-1][2])/distancia(pd[0],pd[1],pd[2],pd[3]))

                    if Sim>=s:
                        clus.extend(Sc)
        if len(clus)>0:
            Cluster.append(clus)
        #break
    print(count)
    return Cluster

In [16]:
T = netscan_phase2(C,trajetorias,0.5)

1806


In [70]:
for i in range(raiz):
    for j in range(raiz):
        if n[i][j]<0:
            n[i][j]= -1*n[i][j]
print(n)

[[ 2  2  2 ...  7  1  8]
 [ 1  1 13 ...  7  6  1]
 [ 8  1  2 ...  3  1  3]
 ...
 [ 5  5  5 ...  1  1  1]
 [ 1  1  1 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


In [17]:
print(len(C))
print(len(T))
print(T[0][0])
print(C[0][0])

111
108
(2, datetime.datetime(2008, 2, 3, 8, 24, 26, 73000), 23871, 116.448722216086, 39.9560374756762)
(23871, 116.448722216086, 39.9560374756762)
